In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path=r'C:\Users\erosb\08-2023 Instacart Basket Analysis'

# Task 4.8

### #1 import dataframe

In [3]:
ords_prods_merge=pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged3.pkl'))

### #2 find aggregated mean order number by department 

In [4]:
ords_prods_merge.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### #3 analyze result

In the subset, the highest average order number was in dept. 19 (snacks) and the lowest in 17 (household). In the entire dataframe, the highest average is in dept. 21 (missing). The next highest is 10 (bulk). The lowest is dept. 5 (alcohol), second lowest is 8 (pets).

### #4 create loyalty flag

In [5]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [6]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [7]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [8]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 10), 'loyalty_flag'] = 'New customer'

### #5 check stats of product prices for each loyalty category

In [9]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

The mean price of products purchased by loyal customers is lower than that of new and regular customers. The min/max are all the same due to limits applied by data engineers.

### #6 create spending flag

In [11]:
#get average price spent on a product by user
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [13]:
#check
ords_prods_merge.head(20)

,order_id,user_id,order_number,orders_day_of_week,hour_order_placed,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,aisle_id,department_id,prices,_exist,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,avg_price
0,2539329,1,1,2,8,NaN,196,1,0,both,...,77,7,9.0,both,Mid range product,Regularly busy,Average orders,10,New customer,6.367797
1,2398795,1,2,3,7,15.0,196,1,1,both,...,77,7,9.0,both,Mid range product,Slowest days,Average orders,10,New customer,6.367797
2,473747,1,3,3,12,21.0,196,1,1,both,...,77,7,9.0,both,Mid range product,Slowest days,Most orders,10,New customer,6.367797
3,2254736,1,4,4,7,29.0,196,1,1,both,...,77,7,9.0,both,Mid range product,Slowest days,Average orders,10,New customer,6.367797
4,431534,1,5,4,15,28.0,196,1,1,both,...,77,7,9.0,both,Mid range product,Slowest days,Most orders,10,New customer,6.367797
5,3367565,1,6,2,7,19.0,196,1,1,both,...,77,7,9.0,both,Mid range product,Regularly busy,Average orders,10,New customer,6.367797
6,550135,1,7,1,9,20.0,196,1,1,both,...,77,7,9.0,both,Mid range product,Busiest days,Most orders,10,New customer,6.367797
7,3108588,1,8,1,14,14.0,196,2,1,both,...,77,7,9.0,both,Mid range product,Busiest days,Most orders,10,New customer,6.367797
8,2295261,1,9,1,16,0.0,196,4,1,both,...,77,7,9.0,both,Mid range product,Busiest days,Most orders,10,New customer,6.367797
9,2550362,1,10,4,8,30.0,196,1,1,both,...,77,7,9.0,both,Mid range product,Slowest days,Average orders,10,New customer,6.367797


In [14]:
ords_prods_merge.loc[ords_prods_merge['avg_price']<10, 'spending_flag'] = 'Low spender'

In [15]:
ords_prods_merge.loc[ords_prods_merge['avg_price']>=10, 'spending_flag'] = 'High spender'

In [16]:
#check frequency
ords_prods_merge['spending_flag'].value_counts(dropna=False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

### #7 determine frequent customers

In [18]:
#new column for median days since prior order
ords_prods_merge['med_dspo'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [19]:
ords_prods_merge.head(20)

,order_id,user_id,order_number,orders_day_of_week,hour_order_placed,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,prices,_exist,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,med_dspo
0,2539329,1,1,2,8,NaN,196,1,0,both,...,9.0,both,Mid range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5
1,2398795,1,2,3,7,15.0,196,1,1,both,...,9.0,both,Mid range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5
2,473747,1,3,3,12,21.0,196,1,1,both,...,9.0,both,Mid range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5
3,2254736,1,4,4,7,29.0,196,1,1,both,...,9.0,both,Mid range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5
4,431534,1,5,4,15,28.0,196,1,1,both,...,9.0,both,Mid range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5
5,3367565,1,6,2,7,19.0,196,1,1,both,...,9.0,both,Mid range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5
6,550135,1,7,1,9,20.0,196,1,1,both,...,9.0,both,Mid range product,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5
7,3108588,1,8,1,14,14.0,196,2,1,both,...,9.0,both,Mid range product,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5
8,2295261,1,9,1,16,0.0,196,4,1,both,...,9.0,both,Mid range product,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5
9,2550362,1,10,4,8,30.0,196,1,1,both,...,9.0,both,Mid range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5


In [20]:
#frequency flag
ords_prods_merge.loc[ords_prods_merge['med_dspo'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [25]:
ords_prods_merge.loc[(ords_prods_merge['med_dspo']>10) & (ords_prods_merge['med_dspo']<=20), 'frequency_flag'] = 'Regular customer'

In [26]:
ords_prods_merge.loc[ords_prods_merge['med_dspo']<=10, 'frequency_flag'] = 'Frequent customer'

In [31]:
ords_prods_merge['frequency_flag'].value_counts(dropna=False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

In [ ]:
ords_prods_merge.to_pickle(os.path.join)